# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298420502471                   -0.85    5.1
  2   -8.300197304515       -2.75       -1.25    1.8
  3   -8.300446165257       -3.60       -1.88    2.5
  4   -8.300462214547       -4.79       -2.72    1.5
  5   -8.300464200718       -5.70       -3.08    5.9
  6   -8.300464416856       -6.67       -3.21    8.5
  7   -8.300464538111       -6.92       -3.36    1.0
  8   -8.300464585749       -7.32       -3.47    2.0
  9   -8.300464631168       -7.34       -3.68    1.0
 10   -8.300464635860       -8.33       -3.78    1.0
 11   -8.300464642805       -8.16       -4.09    1.5


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67453660738                   -0.70    6.6
  2   -16.67859062541       -2.39       -1.14    1.4
  3   -16.67922217564       -3.20       -1.86    4.0
  4   -16.67927698592       -4.26       -2.75    3.9
  5   -16.67928601300       -5.04       -3.18    4.6
  6   -16.67928621554       -6.69       -3.49    3.5
  7   -16.67928622203       -8.19       -3.97    1.0
  8   -16.67928622423       -8.66       -4.59    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32728774717                   -0.56    7.0
  2   -33.33452360122       -2.14       -1.00    1.5
  3   -33.33598471993       -2.84       -1.72    6.9
  4   -33.33678047388       -3.10       -2.20    4.9
  5   -33.33691838941       -3.86       -2.62   10.4
  6   -33.33694378742       -4.60       -3.72    2.4
  7   -33.33694386351       -7.12       -3.90    4.6
  8   -33.33694392612       -7.20       -4.75    1.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298492349478                   -0.85    5.1
  2   -8.300256622479       -2.75       -1.59    1.0
  3   -8.300446636303       -3.72       -2.52    4.6
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│     100
│       4
│       3
│       5
│       4
│       3
│       3
│       3
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:34
  4   -8.300360790130   +   -4.07       -2.25   15.6
  5   -8.300464566143       -3.98       -3.78    2.8
  6   -8.300464610894       -7.35       -4.04    3.4


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32561418477                   -0.56    7.1
  2   -33.32984248559       -2.37       -1.28    1.0
  3   -27.83673478666   +    0.74       -0.70    7.4
  4   -33.14097358889        0.72       -1.41    6.4
  5   -33.28772918118       -0.83       -1.42    4.1
  6   -33.22312122931   +   -1.19       -1.52    4.1
  7   -33.07208356206   +   -0.82       -1.35    5.0
  8   -33.33299040532       -0.58       -2.21    4.6
  9   -33.33362974661       -3.19       -2.17    2.2
 10   -33.33619250605       -2.59       -2.49    1.9
 11   -33.33655233360       -3.44       -2.69    2.0
 12   -33.33686003552       -3.51       -2.98    3.0
 13   -33.33692536727       -4.18       -3.28    3.0
 14   -33.33694142912       -4.79       -3.61    2.9
 15   -33.33694161987       -6.72       -3.87    3.1
 16   -33.33694390674       -5.64       -4.68    2.8


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.